# Convolutional Neural Network

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Importing the libraries

In [ ]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import VGG19
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense


In [ ]:
!pip install tensorflow


In [ ]:
!pip install keras



In [ ]:
!pip install keras-tuner

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 3.3 MB/s eta 0:00:00


In [ ]:
tf.__version__

'2.15.0'

## Part 1 - Data Preprocessing

### Preprocessing the Training set

In [ ]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)
training_set = train_datagen.flow_from_directory('/content/drive/MyDrive/Images',
                                                 target_size = (64, 64),
                                                 batch_size = 32,
                                                 class_mode = 'binary')


Found 5918 images belonging to 2 classes.


### Preprocessing the Test set

In [ ]:
test_datagen = ImageDataGenerator(rescale = 1./255)
test_set = test_datagen.flow_from_directory('/content/drive/MyDrive/Test',
                                            target_size = (64, 64),
                                            batch_size = 32,
                                            class_mode = 'binary')

Found 64 images belonging to 2 classes.


## Part 2 - Building and Initialising the CNN

In [ ]:

def create_model(learning_rate=0.001, num_filters=32, filter_size=3):
  model = Sequential()
  model.add(Conv2D(num_filters, filter_size, activation='relu', kernel_initializer='he_uniform', input_shape=(64, 64, 3)))
  model.add(MaxPooling2D(pool_size=(2, 2)))

  model.add(Flatten())
  model.add(Dense(128, activation='relu', kernel_initializer='he_uniform'))
  model.add(Dense(1, activation='sigmoid'))

  model.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate), metrics=['accuracy'])
  return model

In [ ]:
base_model = VGG19(weights='imagenet', include_top=False, input_shape=(64, 64, 3))


80134624/80134624 [==============================] - 0s 0us/step


In [ ]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)

model = Model(inputs=base_model.input, outputs=predictions)


In [ ]:
for layer in base_model.layers:
    layer.trainable = False


In [ ]:
model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])


In [ ]:
history = model.fit(x = training_set,validation_data = test_set, epochs = 5)


Epoch 1/5
185/185 [==============================] - 1278s 6s/step - loss: 0.7097 - accuracy: 0.5046 - val_loss: 0.7058 - val_accuracy: 0.5000
Epoch 2/5
185/185 [==============================] - 407s 2s/step - loss: 0.7056 - accuracy: 0.4943 - val_loss: 0.6947 - val_accuracy: 0.5000
Epoch 3/5
185/185 [==============================] - 407s 2s/step - loss: 0.7012 - accuracy: 0.5020 - val_loss: 0.6961 - val_accuracy: 0.5000


KeyboardInterrupt: 

## Part 4 - Making a single prediction

In [ ]:
import numpy as np
from keras.preprocessing import image
test_image = image.load_img('/content/drive/MyDrive/singlePrediction/Copy of V_12-0110.jpg', target_size = (64, 64))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = model.predict(test_image)
training_set.class_indices
if result[0][0] == 1:
  prediction = 'violence'
else:
  prediction = 'non violence'

1/1 [==============================] - 0s 92ms/step


In [ ]:
print(prediction)

non violence
